In [1]:
%cd summac

D:\msc_work\sumVeracity\summac\summac


In [ ]:
from summac.benchmark import SummaCBenchmark
# import summac.run_baseline as utils_summac_benchmark
# import random
import warnings
warnings.filterwarnings("ignore")

# benchmark = SummaCBenchmark(benchmark_folder="summac/data/summac_benchmark/", cut="test")
# benchmark.print_stats()

# Table 2: Main Table of Results


In [ ]:
import sklearn, torch, numpy as np, json, os, tqdm, pandas as pd, nltk, seaborn as sns
from summac.model_guardrails import NERInaccuracyPenalty
from summac.model_summac import SummaCConv, SummaCZS
from summac.model_baseline import BaselineScorer
# from model_entailment import EntailmentScorer
# from model_classifier import Classifier
from summac.utils_scoring import ScorerWrapper

use_cache = True
scorers = [
	{"name": "NER", "model": NERInaccuracyPenalty(), "only_doc": True, "sign": 1},
	#     {"name": "MNLI", "model": EntailmentScorer(model_card="roberta-large-mnli", contradiction_idx=0), "sign": 1},
	# {"name": "FactCC-CLS", "model": Classifier(model_card="roberta-base", score_class=1, model_file="/home/phillab/models/cls_roberta-base_factcc_first_0_f1_0.4766.bin"), "sign": 1, "only_doc": True},
	{"name": "DAE", "model": BaselineScorer(model="dae"), "only_doc": True, "sign": 1},
	{"name": "FEQA", "model": BaselineScorer(model="feqa"), "only_doc": True, "sign": 1},
	{"name": "QuestEval", "model": BaselineScorer(model="questeval"), "only_doc": True, "sign": 1},
	{"name": "SummaC-ZS-VITC-L",
	 "model": SummaCZS(granularity="sentence", model_name="vitc", imager_load_cache=use_cache), "sign": 1,
	 "only_doc": True},
	# {"name": "SummaC-Histo-VITC-L", "model": SummaCConv(models=["vitc"], granularity="sentence", start_file="/home/phillab/models/summac/vitc_sentence_percentile_e_bacc0.744.bin", bins="percentile", imager_load_cache=use_cache, device="cpu"), "sign": 1, "only_doc": True},
]

scorer_doc = ScorerWrapper(scorers, scoring_method="sum", max_batch_size=20, use_caching=True)
scorer_para = ScorerWrapper([s for s in scorers if "only_doc" not in s], scoring_method="sum", max_batch_size=20,
							use_caching=True)

In [ ]:
from summac import run_baseline, utils_scorer

results = []
for dataset in benchmark.datasets:
	print("======= %s ========" % (dataset["name"]))
	datas = dataset["task"]
	run_baseline.compute_doc_level(datas)
	utils_scorer.compute_paragraph_level(scorer_para, datas)

	labels = [d["label"] for d in datas]
	pred_labels = [k for k in datas[0].keys() if "pred_" in k]
	for pred_label in pred_labels:
		model_name, input_type = pred_label.replace("pred_", "").split("|")
		preds = [d[pred_label] for d in datas]
		scores = [d[pred_label.replace("pred_", "")] for d in datas]
		balanced_acc = sklearn.metrics.balanced_accuracy_score(labels, preds)
		roc_auc = sklearn.metrics.roc_auc_score(labels, scores)

		results.append({"model_name": model_name, "dataset_name": dataset["name"],
						"input": input_type, "%s_bacc" % (dataset["name"]): balanced_acc,
						"%s_roc_auc" % (dataset["name"]): roc_auc,
						"labels": labels, "preds": preds, "scores": scores})

Found cached dataset xsum (C:/Users/Aviv/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71)


  0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

In [7]:
# If needed for caching
for scorer in scorers:
	if "SummaC" in scorer["name"]:
		scorer["model"].save_imager_cache()

NameError: name 'scorers' is not defined

## Balanced accuracy score

In [ ]:
cm = sns.light_palette("green", as_cmap=True)


def highlight_max(data):
	is_max = data == data.max()
	return ['font-weight: bold' if v else '' for v in is_max]


df = pd.DataFrame(results)
df = df.groupby(["model_name", "input"]).agg({"%s_bacc" % (d): "mean" for d in benchmark.task_name_to_task})
df.rename(columns={k: k.replace("_bacc", "") for k in df.keys()}, inplace=True)
df.drop("total", inplace=True)
df["overall"] = (df["factcc"] + df["frank"] + df["polytope"] + df["cogensumm"] + df["summeval"] + df["xsumfaith"]) / (
	6.0)

df.style.apply(highlight_max).background_gradient(cmap=cm, high=1.0, low=0.0).set_precision(3).set_caption(
	"Balanced Accuracy")

In [ ]:
import random

# Analysis with confidence interval
strongest_baseline = {"cogensumm": "DAE", "xsumfaith": "NER", "polytope": "QuestEval", "factcc": "DAE",
					  "summeval": "QuestEval", "frank": "QuestEval"}

P5 = 5 / 2  # Correction due to the fact that we are running 2 tests with the same data
P1 = 1 / 2  # Correction due to the fact that we are running 2 tests with the same data


def resample_balanced_acc(preds, labels, n_samples=100, sample_ratio=0.7):
	N = len(preds)
	idxs = list(range(N))
	N_batch = int(sample_ratio * N)

	bal_accs = []
	for _ in range(n_samples):
		random.shuffle(idxs)
		batch_preds = [preds[i] for i in idxs[:N_batch]]
		batch_labels = [labels[i] for i in idxs[:N_batch]]

		bal_accs.append(sklearn.metrics.balanced_accuracy_score(batch_labels, batch_preds))
	return bal_accs


print("DATASET NAME".ljust(15), "MODEL NAME".ljust(20))

sampled_batch_preds = {res["model_name"]: [] for res in results}
for res in results:
	if res["model_name"] == "total":
		print("==================================================")
		continue

	samples = resample_balanced_acc(res["preds"], res["labels"])
	sampled_batch_preds[res["model_name"]].append(samples)
	low5, high5 = np.percentile(samples, P5), np.percentile(samples, 100 - P5)
	low1, high1 = np.percentile(samples, P1), np.percentile(samples, 100 - P1)
	bacc = sklearn.metrics.balanced_accuracy_score(res["labels"], res["preds"])
	if "SummaC" in res["model_name"] or res["model_name"] == strongest_baseline[res["dataset_name"]]:

		print(res["dataset_name"].ljust(15), res["model_name"].ljust(20),
			  " - %.3f (%.3f - %.3f) (%.3f - %.3f)" % (bacc, low5, high5, low1, high1))
		if res["model_name"] == strongest_baseline[res["dataset_name"]]:
			bl5, bh5, bl1, bh1 = low5, high5, low1, high1
			print("--------------")
		else:
			if low5 >= bh5:
				print("Significant difference (p < 0.05)")
			if low1 >= bh1:
				print("Significant difference (p < 0.01)")

print("==========================")
print("==========================")
print("==========================")

# baseline = np.mean(np.array(sampled_batch_preds["QuestEval"]), axis=0)
summaczs = np.mean(np.array(sampled_batch_preds["SummaC-ZS-VITC-L"]), axis=0)
summacconv = np.mean(np.array(sampled_batch_preds["SummaC-Histo-VITC-L"]), axis=0)

for model in ["QuestEval", "SummaC-ZS-VITC-L", "SummaC-Histo-VITC-L"]:
	samples = np.mean(np.array(sampled_batch_preds[model]), axis=0)
	low5, high5 = np.percentile(samples, P5), np.percentile(samples, 100 - P5)
	low1, high1 = np.percentile(samples, P1), np.percentile(samples, 100 - P1)

	print("OVERALL".ljust(15), model.ljust(20), " - (%.3f - %.3f) (%.3f - %.3f)" % (low5, high5, low1, high1))

## ROC AUC score

In [ ]:
df = pd.DataFrame(results)
df = df.groupby(["model_name", "input"]).agg({"%s_roc_auc" % (d): "mean" for d in benchmark.task_name_to_task})
df.rename(columns={k: k.replace("_roc_auc", "") for k in df.keys()}, inplace=True)
df.drop("total", inplace=True)
df["overall"] = (df["factcc"] + df["frank"] + df["polytope"] + df["cogensumm"] + df["summeval"] + df["xsumfaith"]) / (
	6.0)

df.style.apply(highlight_max).background_gradient(cmap=cm, high=1.0, low=0.0).set_precision(3).set_caption("ROC AUC")

In [ ]:
# Analysis with confidence interval
strongest_baseline = {"cogensumm": "DAE", "xsumfaith": "QuestEval", "polytope": "QuestEval", "factcc": "DAE",
					  "summeval": "QuestEval", "frank": "QuestEval"}

P5 = 5 / 2  # Correction due to the fact that we are running 2 tests with the same data
P1 = 1 / 2  # Correction due to the fact that we are running 2 tests with the same data


def resample_roc_auc(scores, labels, n_samples=100, sample_ratio=0.7):
	N = len(scores)
	idxs = list(range(N))
	N_batch = int(sample_ratio * N)

	roc_aucs = []
	for _ in range(n_samples):
		random.shuffle(idxs)
		batch_scores = [scores[i] for i in idxs[:N_batch]]
		batch_labels = [labels[i] for i in idxs[:N_batch]]
		roc_aucs.append(sklearn.metrics.roc_auc_score(batch_labels, batch_scores))
	return roc_aucs


sampled_batch_preds = {res["model_name"]: [] for res in results}
print("DATASET NAME".ljust(15), "MODEL NAME".ljust(20))
for res in results:
	if res["model_name"] == "total":
		print("==================================================")
		continue
	samples = resample_roc_auc(res["scores"], res["labels"])
	sampled_batch_preds[res["model_name"]].append(samples)
	low5, high5 = np.percentile(samples, P5), np.percentile(samples, 100 - P5)
	low1, high1 = np.percentile(samples, P1), np.percentile(samples, 100 - P1)
	roc_auc = sklearn.metrics.roc_auc_score(res["labels"], res["scores"])
	if "SummaC" in res["model_name"] or res["model_name"] == strongest_baseline[res["dataset_name"]]:
		print(res["dataset_name"].ljust(15), res["model_name"].ljust(20),
			  " - %.3f (%.3f - %.3f) (%.3f - %.3f)" % (roc_auc, low5, high5, low1, high1))
		if res["model_name"] == strongest_baseline[res["dataset_name"]]:
			bl5, bh5, bl1, bh1 = low5, high5, low1, high1
			print("--------------")
		else:
			if low5 >= bh5:
				print("Significant difference (p < 0.05)")
			if low1 >= bh1:
				print("Significant difference (p < 0.01)")

print("==========================")
print("==========================")
print("==========================")

baseline = np.mean(np.array(sampled_batch_preds["QuestEval"]), axis=0)
summaczs = np.mean(np.array(sampled_batch_preds["SummaC-ZS-VITC-L"]), axis=0)
summacconv = np.mean(np.array(sampled_batch_preds["SummaC-Histo-VITC-L"]), axis=0)

for model in ["QuestEval", "SummaC-ZS-VITC-L", "SummaC-Histo-VITC-L"]:
	samples = np.mean(np.array(sampled_batch_preds[model]), axis=0)
	low5, high5 = np.percentile(samples, P5), np.percentile(samples, 100 - P5)
	low1, high1 = np.percentile(samples, P1), np.percentile(samples, 100 - P1)

	print("OVERALL".ljust(15), model.ljust(20), " - (%.3f - %.3f) (%.3f - %.3f)" % (low5, high5, low1, high1))